# Практическая работа №4 «Контроль целостности (биты четности, контрольные цифры, CRC и ECC)»
В практической работе необходимо определить контрольные данные с использованием следующих способов:
- битов четности. В качестве исходных данных принять битовое представление букв фамилии в соответствии с кодировкой Windows 1251 (табл.6.2);
- контрольных цифр. В качестве исходных данных принять необходимое количество цифр (за исключением контрольной) из строки, состоящей из кодов букв фамилии, имени и отчества согласно их положению в алфавите:
- по алгоритму Луна (15 цифр);
- для штрихкода по стандарту EAN-13 (12 цифр);
- для ИНН физического лица (10 цифр);
- для кодов станций на железнодорожном транспорте (5 цифр);
- контрольных сумм (CRC). В качестве исходных данных принять коды 1-ой, 2-ой и 3-ей буквы своей фамилии согласно их положению в алфавите; порождающего полинома - G(x) = x4 + x1 + x0.
- кода коррекции ошибок (ECC). В качестве исходных данных принять первые 11 битов первых двух буквы своей фамилии в соответствии с кодировкой Windows 1251 (табл.6.2). Рассчитать вектор контрольных битов и вектора синдромов при отсутствии ошибки, одиночной и двойной ошибке.
При оформлении отчета необходимо привести необходимые таблицы, исходные данные, расчеты и результаты. Отчет прикрепить в edu.susu.ru.


In [1]:
import json

import numpy as np
import pandas as pd

In [2]:
with open('instance/settings.json') as f:
    settings = json.load(f)
    LAST_NAME = settings['last_name']
    FIRST_NAME = settings['first_name']
    PATRONYMIC = settings['patronymic']

## Проверка четности

In [3]:
bits = ['{0:0>8b}'.format(c) for c in LAST_NAME.encode('cp1251')]

In [4]:
source = {
#     'Символ': [c for c in LAST_NAME],
#     'Код символа': [c for c in LAST_NAME.encode('cp1251')],
    'Битовая строка': bits
}

In [5]:
data = pd.DataFrame(source)

In [6]:
data['Паритетный бит', 'четный (even)'] = data['Битовая строка'].apply(lambda x: sum(map(int, x)) & 1)
data['Паритетный бит', 'нечетный (odd)'] = data['Битовая строка'].apply(lambda x: 1 - (sum(map(int, x)) & 1))

In [7]:
data.columns = pd.MultiIndex.from_tuples([
#     ('', 'Символ'),
#     ('', 'Код символа'),
    ('', 'Битовая строка'),
    ('Паритетный бит', 'четный (even)'), 
    ('Паритетный бит', 'нечетный (odd)'),
])

In [8]:
data

Паритетный бит               
  Битовая строка  четный (even) нечетный (odd)
0       01100001              1              0
1       01101100              0              1
2       01101011              1              0
3       01100001              1              0
4       01110000              1              0
5       01101111              0              1
6       01110110              1              0

## Использование контрольных цифр

In [9]:
digits = list(map(int, ''.join(map(str, (LAST_NAME + FIRST_NAME + PATRONYMIC).encode('cp1251')))))

### Алгоритм Луна

In [10]:
luna_input = digits[:15]

In [11]:
if (len(luna_input) & 1) == 1:
    luna_input = [0] + luna_input

{{'$input = [%s]$' % ','.join(map(str, luna_input))}}

Каждая из цифр, стоящая в нечетной позиции, умножается на 2, после чего вычисляется остаток от деления на 9.

In [12]:
e_odd = list(map(lambda x: (x * 2) % 9, luna_input[1::2]))
md = '$E_n = [%s]$' % ','.join(map(str, e_odd))

{{md}}

Вычисляется сумма остатков $S_{odd}$.

In [13]:
s_odd = sum(e_odd)
md = f'$S_{{odd}} = {s_odd}$'

{{md}}

Вычисляется сумма цифр $S_{even}$, стоящих в четных позициях, за исключением последней.

In [14]:
s_even = sum(luna_input[:-1:2])
md = f'$S_{{even}} = {s_even}$'

{{md}}

Вычисляется контрольная (последняя) цифра $cd$ из уравнения $(S_{odd} + S_{even} + {cd}) \bmod 10 = 0$.

In [15]:
cd = next(i for i in range(10) if ((s_odd + s_even + i) % 10) == 0)

{{f'$cd = {cd}$'}}

### Штрихкод по стандарту EAN-13

In [16]:
ean_input = digits[:13]

Вычисляется сумма цифр $S_{odd}$, стоящих в нечетных позициях, за исключением последней (нумерация с 1)

In [17]:
s_odd = sum(ean_input[:-1:2])
md = f'$S_{{odd}} = {s_odd}$'

{{md}}

Вычисляется утроенная сумма цифр $S_{even}$, стоящих в четных позициях.

In [18]:
s_even = 3 * sum(ean_input[1::2])
md = f'$S_{{even}} = {s_even}$'

{{md}}

Вычисляется контрольная (последняя) цифра $cd$ из уравнения $(S_{odd} + S_{even} + {cd}) \bmod 10 = 0$.

In [19]:
cd = next(i for i in range(10) if ((s_odd + s_even + i) % 10) == 0)

{{f'$cd = {cd}$'}}

### Индивидуальный номер налогоплательщика

In [20]:
tin_input = digits[:12]

Для десятизначного ИНН юридического лица:  
$n_{10} = ((2n_1 + 4n_2 + 10n_3 + 3n_4 + 5n_5 + 9n_6 + 4n_7 + 6n_8 + 8n_9) \bmod 11) \bmod 10$

In [21]:
def get_legal_entity_cd(nums):
    coefs = [2, 4, 10, 3, 5, 9, 4, 6, 8]
    return (sum([c*n for n, c in zip(nums[:9], coefs)]) % 11) % 10

In [22]:
n_10 = get_legal_entity_cd(tin_input[:10])
md = f'$n_{{10}} = {n_10}$'

{{md}}

Для двенадцатизначного ИНН физического лица:  
$n_{11} = ((7n_1 + 2n_2 + 4n_3 + 10n_4 + 3n_5 + 5n_6 + 9n_7 + 4n_8 + 6n_9 + 8n_{10}) \bmod 11) \bmod 10$
$n_{12} = ((3n_1 + 7n_2 + 2n_3 + 4n_4 + 10n_5 + 3n_6 + 5n_7 + 9n_8 + 4n_9 + 6n_{10} + 8n_{11}) \bmod 11) \bmod 10$

In [23]:
def get_individual_cd(nums):
    coefs_11 = [7, 2, 4, 10, 3, 5, 9, 3, 6, 8]
    coefs_12 = [3] + coefs_11
    n_11 = (sum([c*n for n, c in zip(nums[:10], coefs_11)]) % 11) % 10
    n_12 = (sum([c*n for n, c in zip(nums[:11], coefs_12)]) % 11) % 10
    return n_11, n_12

In [24]:
n_11, n_12 = get_individual_cd(tin_input)
md = f'$n_{{11}} = {n_11}, n_{{12}} = {n_12}$'

{{md}}

### Коды станций на железнодорожном транспорте

In [25]:
r_input = digits[:6]

Последняя цифра кода (n_6) является контрольной и определяется по следующей формуле:    
$n_6 = (1n_1 + 2n_2 + 3n_3 + 4n_4 + 5n_5) \bmod 11$  
Если остаток от деления меньше $10$, то он является контрольной цифрой, иначе выполняют сдвиг весового ряда на две позиции и вычисления повторяют:  
$n_6 = (3n_1 + 4n_2 + 5n_3 + 6n_4 + 7n_5) \bmod 11$  
Если новый остаток от деления вновь получится равным $10$, то контрольная цифра принимается равной $0$, иначе - остатку

In [26]:
def get_r_cd(nums):
    coefs_success = [1, 2, 3, 4, 5]
    coefs_fail = [3, 4, 5, 6, 7]
    n_6 = sum([c*n for n, c in zip(nums[:5], coefs_success)]) % 11
    if n_6 != 10:
        return n_6
    n_6 = sum([c*n for n, c in zip(nums[:5], coefs_fail)]) % 11
    if n_6 != 10:
        return n_6
    return 0

In [27]:
n_6 = get_r_cd(r_input)
md = f'$n_{{6}} = {n_6}$'

{{md}}

### Использование контрольных сумм

In [28]:
crc_input = ''.join(map(lambda x: '{0:b}'.format(ord(x) - ord('a') + 1), LAST_NAME[:3]))
# crc_input = ''.join(map(lambda x: '{0:0>8b}'.format(ord(x) - ord('a') + 1), LAST_NAME[:3]))
crc_input

'111001011'

In [29]:
def crc_remainder(input_bitstring, polynomial_bitstring, initial_filler = '0'):
    '''
    Calculates the CRC remainder of a string of bits using a chosen polynomial.
    initial_filler should be '1' or '0'.
    '''
    polynomial_bitstring = polynomial_bitstring.lstrip('0')
    len_input = len(input_bitstring)
    initial_padding = initial_filler * (len(polynomial_bitstring) - 1)
    input_padded_array = list(input_bitstring + initial_padding)
    while '1' in input_padded_array[:len_input]:
        cur_shift = input_padded_array.index('1')
        for i in range(len(polynomial_bitstring)):
            input_padded_array[cur_shift + i] = str(int(polynomial_bitstring[i] != input_padded_array[cur_shift + i]))
    return ''.join(input_padded_array)[len_input:]

def crc_check(input_bitstring, polynomial_bitstring, check_value):
    '''
    Calculates the CRC check of a string of bits using a chosen polynomial.
    '''
    polynomial_bitstring = polynomial_bitstring.lstrip('0')
    len_input = len(input_bitstring)
    initial_padding = check_value
    input_padded_array = list(input_bitstring + initial_padding)
    while '1' in input_padded_array[:len_input]:
        cur_shift = input_padded_array.index('1')
        for i in range(len(polynomial_bitstring)):
            input_padded_array[cur_shift + i] = str(int(polynomial_bitstring[i] != input_padded_array[cur_shift + i]))
    return ('1' not in ''.join(input_padded_array)[len_input:])

In [30]:
g_string = '10011' # x^4 + 0*x^3 + 0*x^2 + x + x^0

In [31]:
crc_remainder(crc_input, g_string)

'1000'

### ECC

In [32]:
ecc_input = ''.join(['{0:0>8b}'.format(c) for c in LAST_NAME.encode('cp1251')[:2]])[:11]

In [33]:
ecc_input = list(map(int, ecc_input))

In [34]:
def gen_bit_pos(num):
    return np.arange(1, num + 1)

In [35]:
def gen_bit_names(num):
    return ['x{}'.format(i) for i in gen_bit_pos(num)]

In [36]:
source = pd.DataFrame({
    'Номер позиции бита': gen_bit_pos(11),
    'Обозначение бита': gen_bit_names(11),
    'Значение бита': list(ecc_input),
})
source = source.set_index('Номер позиции бита')
source.T

Номер позиции бита,1,2,3,4,5,6,7,8,9,10,11
Обозначение бита,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11
Значение бита,0,1,1,0,0,0,0,1,0,1,1


In [37]:
def is_power_of_2(num):
    return num != 0 and ((num & (num - 1)) == 0)

In [38]:
def gen_bit_pos_r(num):
    cnt = sum(1 for i in range(1, num + 1) if is_power_of_2(i))
    return np.arange(1, num + cnt + 1)

In [39]:
def gen_bit_names_r(num):
    res = []
    r = 1
    x = 1
    for i in gen_bit_pos_r(num):
        if is_power_of_2(i):
            res.append('r{}'.format(r))
            r += 1
        else:
            res.append('x{}'.format(x))
            x += 1
    return res

In [40]:
def get_r_values(num):
    r_values = np.array([list(map(int, list('{0:0>4b}'.format(n))[::-1])) for n in gen_bit_pos_r(num)]).T
    r_names = ['r{}'.format(1 + x) for x in range(sum(1 for i in range(1, num + 1) if is_power_of_2(i)))]
    return {k:v for k, v in zip(r_names, r_values)}

In [41]:
def expand_bit_value(ecc):
    p = ecc[:]
    num = len(ecc)
    res = []
    for i in gen_bit_pos_r(num):
        if is_power_of_2(i):
            res.append(0)
        else:
            res.append(int(p[0]))
            p = p[1:]
    return res

In [42]:
def get_r_bits(ecc):
    a = expand_bit_value(list(ecc_input))
    b = [list(map(int, list('{0:0>4b}'.format(n))[::-1])) for n in gen_bit_pos_r(len(ecc))]
    return np.mod(np.matmul(a, b), 2)

In [43]:
def fill_r_bits(ecc, rbits=None):
    res = expand_bit_value(ecc)
    rbits = rbits or get_r_bits(ecc)
    for i in range(len(res)):
        if is_power_of_2(i + 1):
            res[i] = rbits[0]
            rbits = rbits[1:]
    return res

In [44]:
ecc_df = pd.DataFrame({
    'Номер позиции бита': gen_bit_pos_r(11),
    'Обозначение бита': gen_bit_names_r(11),
    'Значение бита': fill_r_bits(ecc_input),
    **get_r_values(11)
}).set_index('Номер позиции бита').T
ecc_df

Номер позиции бита,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
Обозначение бита,r1,r2,x1,r3,x2,x3,x4,r4,x5,x6,x7,x8,x9,x10,x11
Значение бита,0,1,0,1,1,1,0,1,0,0,0,1,0,1,1
r1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1
r2,0,1,1,0,0,1,1,0,0,1,1,0,0,1,1
r3,0,0,0,1,1,1,1,0,0,0,0,1,1,1,1
r4,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1


In [45]:
ecc_df['Pb'] = ['', np.sum(ecc_df.loc['Значение бита']) % 2, '', '', '', '']
ecc_df

Номер позиции бита,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,Pb
Обозначение бита,r1,r2,x1,r3,x2,x3,x4,r4,x5,x6,x7,x8,x9,x10,x11,
Значение бита,0,1,0,1,1,1,0,1,0,0,0,1,0,1,1,0
r1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,
r2,0,1,1,0,0,1,1,0,0,1,1,0,0,1,1,
r3,0,0,0,1,1,1,1,0,0,0,0,1,1,1,1,
r4,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,


In [46]:
check_values_idx = [0, 1, 3, 7, -1]
check = ecc_df.loc['Значение бита'].values[[0, 1, 3, 7, -1]]
ecc_input, check

([0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1], array([0, 1, 1, 1, 0], dtype=object))

In [47]:
ecc_df.shape

(6, 16)

In [48]:
def get_matrix_with_sindrome(ecc, check_bits=None):
    num = len(ecc)
    df = pd.DataFrame({
        'Номер позиции бита': gen_bit_pos_r(num),
        'Обозначение бита': gen_bit_names_r(num),
        'Значение бита': fill_r_bits(ecc_input, check_bits),
        **get_r_values(num)
    }).set_index('Номер позиции бита').T
    pbv = np.sum(df.loc['Значение бита']) % 2
    pb = ['' for i in range(df.shape[0])]
    if check_bits is None:
        pb[1] = pbv
    else:
        pb[1] = check_bits[-1]
    df['Pb'] = pb
    if check_bits is not None:
        snames = ['' for i in range(df.shape[0])]
        svals = ['' for i in range(df.shape[0])]
        cnt = sum(1 for i in gen_bit_names_r(num) if i.startswith('r'))
        snames[1] = 'sp'
        b = [list(map(int, list('{0:0>4b}'.format(n))[::-1])) for n in gen_bit_pos_r(len(ecc))]
        sbits = np.matmul(list(map(int, df.loc['Значение бита'].values[:-1])), b)
        sbits = np.mod(sbits, 2)
        for i in range(2, 2 + cnt):
            snames[i] = 's{}'.format(i - 1)
            svals[i] = sbits[0]
            sbits = sbits[1:]
        svals[1] = pbv
        df['SNames'] = snames
        df['SVals'] = svals
    return df

Просто данные

In [49]:
get_matrix_with_sindrome(ecc_input)

Номер позиции бита,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,Pb
Обозначение бита,r1,r2,x1,r3,x2,x3,x4,r4,x5,x6,x7,x8,x9,x10,x11,
Значение бита,0,1,0,1,1,1,0,1,0,0,0,1,0,1,1,0
r1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,
r2,0,1,1,0,0,1,1,0,0,1,1,0,0,1,1,
r3,0,0,0,1,1,1,1,0,0,0,0,1,1,1,1,
r4,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,


0 синдром

In [50]:
get_matrix_with_sindrome(ecc_input, [0, 1, 1, 1, 0])

Номер позиции бита,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,Pb,SNames,SVals
Обозначение бита,r1,r2,x1,r3,x2,x3,x4,r4,x5,x6,x7,x8,x9,x10,x11,,,
Значение бита,0,1,0,1,1,1,0,1,0,0,0,1,0,1,1,0,sp,0
r1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,,s1,0
r2,0,1,1,0,0,1,1,0,0,1,1,0,0,1,1,,s2,0
r3,0,0,0,1,1,1,1,0,0,0,0,1,1,1,1,,s3,0
r4,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,,s4,0


In [51]:
err1 = ecc_input
err1[-1] = 1 - err1[-1]

In [52]:
get_matrix_with_sindrome(err1, [0, 1, 1, 1, 0])

Номер позиции бита,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,Pb,SNames,SVals
Обозначение бита,r1,r2,x1,r3,x2,x3,x4,r4,x5,x6,x7,x8,x9,x10,x11,,,
Значение бита,0,1,0,1,1,1,0,1,0,0,0,1,0,1,0,0,sp,1
r1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,,s1,1
r2,0,1,1,0,0,1,1,0,0,1,1,0,0,1,1,,s2,1
r3,0,0,0,1,1,1,1,0,0,0,0,1,1,1,1,,s3,1
r4,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,,s4,1


In [53]:
err2 = err1
err2[0] = 1 - err2[0]

In [54]:
get_matrix_with_sindrome(err2, [0, 1, 1, 1, 0])

Номер позиции бита,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,Pb,SNames,SVals
Обозначение бита,r1,r2,x1,r3,x2,x3,x4,r4,x5,x6,x7,x8,x9,x10,x11,,,
Значение бита,0,1,1,1,1,1,0,1,0,0,0,1,0,1,0,0,sp,0
r1,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,,s1,0
r2,0,1,1,0,0,1,1,0,0,1,1,0,0,1,1,,s2,0
r3,0,0,0,1,1,1,1,0,0,0,0,1,1,1,1,,s3,1
r4,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,,s4,1
